<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

`git` tutorial (5)

# ordre et atteignabilité

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

In [ ]:
# si nécessaire, vous pouvez remettre le repository en l'état
# 
# pour cela mettez "true" au lieu de ""
# et bien sûr évaluer la cellule

reset=""

if [ -n "$reset" ]; then 
    cd $TOP
    bash $SCRIPTS/10-my-first-repo.sh
    bash $SCRIPTS/20-my-first-changes.sh
    bash $SCRIPTS/30-my-first-branch.sh
    bash $SCRIPTS/40-kinds-of-merge.sh
fi >& /dev/null

In [ ]:
# si nécessaire, on se place dans le repo git
[ -d my-first-repo ] && cd my-first-repo

pwd

## le point

In [ ]:
# vous devez avoir 9 commits
git log --oneline | wc -l


In [ ]:
# le dernier s'appelle 'conflit résolu'
git log -1 --format="%s"

In [ ]:
# et la branche courante est master
git branch

## liens entre commits

* on a vu qu'un commit est immutable
* ça signifie qu'une fois créé on ne peut pas le changer
* donc par construction un commit ne peut 
  * **que connaitre ses parents**
  * c'-à-d les commits **sur lesquels** il est construit
* et n'a **pas de lien "montant"**
  * vers les commits qui l'utilisent par la suite

<img src="media/order-4-down.png" width="140px">

## atteignabilité

* lorsqu'on fait par exemple `git log` 
  * on part (du commit) de la branche courante
  * on suit par transitivité la relation *parent* 
* il arrive (on a déjà vu des exemples) 
  * qu'avec ce point de vue
  * on ne "voie pas" certains commits
  * qui pourtant sont bien là 

## exemples

In [ ]:
# le point de départ par défaut est HEAD

git l

In [ ]:
# si je donne un point de départ

git l devel

## points de départ usuels

selon les cas (commande CLI, UI, ..) le point de départ est

* souvent (le commit de) la branche courante
* ou une branche qu'on précise sur la ligne de commande
* ou toutes les branches connues (ex. `git log --all`)

In [ ]:
# en partant d'un commit précis

git l HEAD~2

## application

* on tire très souvent parti de cette particularité
* pour "revenir en arrière" ou "récrire l'histoire"
* voyons par exemple comment je pourrais défaire notre dernier merge

pour cela

* il me suffit de faire un `git reset --hard HEAD^`
* la branche master recule d'un cran
* le commit 'conflit résolu' **existe toujours** 
* sauf que 
  * on n'a plus de façon de le trouver à partir des branches connues
  * on peut toujours le retrouver à partir de son sha-1

## revenir en arrière (1)

In [ ]:
git l

In [ ]:
# je fais comme si je notais le sha-1
# du sommet sur un bout de papier

ghost=$(git log -1 --format='%h')
echo $ghost

In [ ]:
# la branche courante est master
# on la fait reculer d'un cran
git reset --hard HEAD^

## revenir en arrière (2)

In [ ]:
# si je regarde master, je ne vois plus devel
git l

In [ ]:
# si je regarde les branches connues
# je ne vois plus le merge 'conflit résolu'
git l --all

## revenir en arrière (3)

à ce stade la plupart des outils (CLI ou UI) ne montrera plus le merge, mais :

In [ ]:
# en précisant le hash
git l $ghost

en pratique:

* ce commit qui n'est **plus accessible à partir des branches** finira vraisemblablement par se faire nottoyer par le *garbage collector*

* sauf action spécifique, ce sera au bout de plusieurs jours..

## défaire le retour en arrière

exercice : pour le sport, comment revenir à la situation initiale ?  
i.e. remettre master sur le commit 'conflit résolu' 

## défaire le retour en arrière : réponse

réponse

* il suffit de merger notre commit `ghost` dans `master`
* c'est bien sûr un fast-forward, inutile de créer un commit 

In [ ]:
# on est toujours dans master
git branch

In [ ]:
git merge $ghost

In [ ]:
git l

## gestion des branches

rappel :

le moyen usuel pour à la fois

* créer une **nouvelle** branche
* et la rendre **courante**

est de faire `git checkout -b nouvelle-branche`

on va voir à présent les commandes pour manipuler les branches de manière plus élémentaire

## `git branch`

la branche courante, c'est celle qui correspond à `HEAD`

In [ ]:
# git branch, sans option
# montre la liste des branches
# la branche courante est mise en relief

git branch

In [ ]:
# git log -1 : pour ne voir que un commit
git log --oneline -1 HEAD

In [ ]:
git l --all

## `git branch` : créer

In [ ]:
# pour juste créer une branche (sans y aller)
# donner simplement un nom et un commit

git branch foobar HEAD~2

git l --all

## `git branch` : renommer

In [ ]:
# pour renommer une branche 
# (même la branche courante d'ailleurs)

git branch -m foobar trucmuche

git l --all

## `git branch` : détruire

In [ ]:
# pour détruire une branche

git branch -d trucmuche

git l --all


on verra plus loin les précautions à prendre

## branche comme signet

* tout à l'heure on avait utilisé un bout de papier
  * pour pouvoir référencer le commit plus tard en cas de souci
* on peut faire la même chose en utilisant une branche comme marque page
* quitte à la détruire ensuite

## branche comme signet

In [ ]:
# je mets un signet
git branch bookmark HEAD

# maintenant si je reset master
git reset --hard HEAD^

In [ ]:
git l --all

## `git branch -D` 

* par contre si j'essaie d'effacer la branche `bookmark`
* git se plaint, car cela rend de nouveau
  * notre commit `conflit résolu' inatteignable 
  * (à partir des branches connues)
* de façon générale, lorsqu'il y a un risque  
  de perdre quelque chose, ça coince

In [ ]:
git branch -d bookmark

In [ ]:
# avec -D on force la destruction
git branch -D bookmark

## pour revenir au point de départ

In [ ]:
# le commit est bien sûr toujours là
git merge $ghost

In [ ]:
git l

## résumé

* chaque commit a connaissance de ses *parents*  
  i.e. ses voisins **vers le bas**

* le lien dans l'autre sens n'est **pas matérialisé**  
  dans un commit (car immutable)

* la partie "visible" ou "atteignable" du repo est ce  
  qu'on peut atteindre **à partir des branches connues**

* lorsqu'un commit n'est plus atteignable
  * il est toujours dans le repo (au - quelques jours)
  * mais il faut son sha-1 pour y accéder